In [ ]:
import pickle
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import punkt
from nltk.corpus.reader import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import requests
from bs4 import BeautifulSoup
import numpy as np
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import dash_renderer
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go
import re

In [ ]:

path_models =  r"Y:\project"
path_svm = path_models + 'best_svc.pickle'
with open(path_svm, 'rb') as data:
    svc_model = pickle.load(data)

In [ ]:
path_tfidf = r"Y:\project\tfidf.pickle"

with open(path_tfidf, 'rb') as data:
    tfidf = pickle.load(data)

In [ ]:
category_codes = {
    'business': 0,
    'entertainment': 1,
    'politics': 2,
    'sport': 3,
    'tech': 4,
    'other':5
}

In [ ]:
def get_news_dailymail():

    
    url = "https://www.dailymail.co.uk"
    
  
    r1 = requests.get(url)
    r1.status_code

    
    coverpage = r1.content

    
    soup1 = BeautifulSoup(coverpage, 'html5lib')

    
    coverpage_news = soup1.find_all('h2', class_='linkro-darkred')
    len(coverpage_news)
    
    number_of_articles = 4

    
    news_contents = []
    list_links = []
    list_titles = []

    for n in np.arange(0, number_of_articles):

        
        link = url + coverpage_news[n].find('a')['href']
        list_links.append(link)

        
        title = coverpage_news[n].find('a').get_text()
        list_titles.append(title)

        
        article = requests.get(link)
        article_content = article.content
        soup_article = BeautifulSoup(article_content, 'html5lib')
        body = soup_article.find_all('p', class_='mol-para-with-font')

        
        list_paragraphs = []
        for p in np.arange(0, len(body)):
            paragraph = body[p].get_text()
            list_paragraphs.append(paragraph)
            final_article = " ".join(list_paragraphs)

        
        final_article = re.sub("\\xa0", "", final_article)
        
        news_contents.append(final_article)
        

   
    df_features = pd.DataFrame(
         {'Content': news_contents 
        })

    
    df_show_info = pd.DataFrame(
        {'Article Title': list_titles,
         'Article Link': list_links,
         'Newspaper': 'Daily Mail'})
    
    return (df_features, df_show_info)

In [ ]:
def get_news_theguardian():
    
    
    url = "https://www.theguardian.com/uk"
    
   
    r1 = requests.get(url)
    r1.status_code

    
    coverpage = r1.content

    
    soup1 = BeautifulSoup(coverpage, 'html5lib')

    
    coverpage_news = soup1.find_all('h3', class_='fc-item__title')
    len(coverpage_news)
    
    number_of_articles = 10

    
    news_contents = []
    list_links = []
    list_titles = []

    for n in np.arange(0, number_of_articles,2):

        
        if "live" in coverpage_news[n].find('a')['href']:  
            continue

       
        link = coverpage_news[n].find('a')['href']
        list_links.append(link)

       
        title = coverpage_news[n].find('a').get_text()
        list_titles.append(title)

        
        article = requests.get(link)
        article_content = article.content
        soup_article = BeautifulSoup(article_content, 'html5lib')
        body = soup_article.find_all('div', class_='content__article-body from-content-api js-article__body')
        x = body[0].find_all('p')

    
        list_paragraphs = []
        for p in np.arange(0, len(x)):
            paragraph = x[p].get_text()
            list_paragraphs.append(paragraph)
            final_article = " ".join(list_paragraphs)

        news_contents.append(final_article)

    
    df_features = pd.DataFrame(
         {'Content': news_contents 
        })


    df_show_info = pd.DataFrame(
        {'Article Title': list_titles,
         'Article Link': list_links,
         'Newspaper': 'The Guardian'})

    
    return (df_features, df_show_info)

In [ ]:
punctuation_signs = list("?:!.,;")
stop_words = list(stopwords.words('english'))

def create_features_from_df(df):
    
    df['Content_Parsed_1'] = df['Content'].str.replace("\r", " ")
    df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("\n", " ")
    df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("    ", " ")
    df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace('"', '')
    
    df['Content_Parsed_2'] = df['Content_Parsed_1'].str.lower()
    
    df['Content_Parsed_3'] = df['Content_Parsed_2']
    for punct_sign in punctuation_signs:
        df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')
        
    df['Content_Parsed_4'] = df['Content_Parsed_3'].str.replace("'s", "")
    
    wordnet_lemmatizer = WordNetLemmatizer()
    nrows = len(df)
    lemmatized_text_list = []
    for row in range(0, nrows):

        
        lemmatized_list = []     
        text = df.loc[row]['Content_Parsed_4']
        text_words = text.split(" ")
        
        for word in text_words:
            lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        lemmatized_text = " ".join(lemmatized_list)
        lemmatized_text_list.append(lemmatized_text)
    
    df['Content_Parsed_5'] = lemmatized_text_list   
    df['Content_Parsed_6'] = df['Content_Parsed_5']
    for stop_word in stop_words:
        regex_stopword = r"\b" + stop_word + r"\b"
        df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')
        
    df = df['Content_Parsed_6']
    df = df.rename(columns={'Content_Parsed_6': 'Content_Parsed'})
    

    features = tfidf.transform(df).toarray()
    
    return features

In [ ]:
def get_category_name(category_id):
    for category, id_ in category_codes.items():    
        if id_ == category_id:
            return category

In [ ]:

def predict_from_features(features):
        

    predictions_proba = svc_model.predict_proba(features).max(axis=1)    
    

    predictions_pre = svc_model.predict(features)


    predictions = []

    for prob, cat in zip(predictions_proba, predictions_pre):
        if prob > .65:
            predictions.append(cat)
        else:
            predictions.append(5)


    categories = [get_category_name(x) for x in predictions]
    
    return categories

In [ ]:

def complete_df(df, categories):
    df['Prediction'] = categories
    return df

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

colors = {
    'background': '#ECECEC',  
    'text': '#696969',
    'titles': '#599ACF',
    'blocks': '#F7F7F7',
    'graph_background': '#F7F7F7',
    'banner': '#C3DCF2'

}


markdown_text1 = '''

This application gathers the latest news from the newspapers  **The Guardian** and **Daily Mail**, predicts their category between **Politics**, **Business**, **Entertainment**, **Sport**, **Tech** and **Other** and then shows a summary.

The scraped news are converted into a numeric feature vector with *TF-IDF vectorization*. Then, a *Support Vector Classifier* is applied to predict each category.

This app is meant for didactic purposes.

Please enter which newspapers would you like to scrape news off and press the **Scrape** button.

'''

markdown_text2 = ""
app.layout = html.Div(style={'backgroundColor':colors['background']}, children=[
    

    html.H1(children=' ',
            style={'padding': '10px'}
           ),
    

    html.Div(
        [
            html.H3(children='News Classification App',
                    style={"margin-bottom": "0px"}
                   ),
            html.H6(children='A Machine Learning based app')
        ],
        style={
            'textAlign': 'center',
            'color': colors['text'],
            'backgroundColor': colors['background']
              },
        className='banner',
            ),
    


    html.H1(children=' ',
            style={'padding': '1px'}),



    html.Div(
        [
            html.Div(
                [
                    html.H6(children='What does this app do?',
                            style={'color':colors['titles']}),
                    
                    html.Div(
                        [dcc.Markdown(children=markdown_text1),],
                        style={'font-size': '12px',
                               'color': colors['text']}),
                                        
                    html.Div(
                        [
                            dcc.Dropdown(
                                options=[
                                    
                                    {'label': 'The Guardian', 'value': 'THG'},
                                    {'label': 'Daily Mail', 'value': 'DML'}
                                        ],
                                value=['EPE'],
                                multi=True,
                                id='checklist'),
                        ],
                        style={'font-size': '12px',
                               'margin-top': '25px'}),
                    
                    html.Div([
                        html.Button('Scrape', 
                                    id='submit', 
                                    type='submit', 
                                    style={'color': colors['blocks'],
                                           'background-color': colors['titles'],
                                           'border': 'None'})],
                        style={'textAlign': 'center',
                               'padding': '20px',
                               "margin-bottom": "0px",
                               'color': colors['titles']}),
            
                    dcc.Loading(id="loading-1", children=[html.Div(id="loading-output-1")], type="circle"),
                    
                    html.Hr(),
                    html.H6(children='Headlines',
                            style={'color': colors['titles']}),

                    
                    html.A(id="textarea1a", target="_blank", style={'font-size': '12px'}),
                    html.P(id="textarea1b", style={'color': colors['text'], 'font-size': '11px'}),
                    html.A(id="textarea2a", target="_blank", style={'font-size': '12px'}),
                    html.P(id="textarea2b", style={'color': colors['text'], 'font-size': '11px'}),
                    html.A(id="textarea3a", target="_blank", style={'font-size': '12px'}),
                    html.P(id="textarea3b", style={'color': colors['text'], 'font-size': '11px'}),
                    html.A(id="textarea4a", target="_blank", style={'font-size': '12px'}),
                    html.P(id="textarea4b", style={'color': colors['text'], 'font-size': '11px'}),
                    html.A(id="textarea5a", target="_blank", style={'font-size': '12px'}),
                    html.P(id="textarea5b", style={'color': colors['text'], 'font-size': '11px'}),
                    html.A(id="textarea6a", target="_blank", style={'font-size': '12px'}),
                    html.P(id="textarea6b", style={'color': colors['text'], 'font-size': '11px'}),
                    html.A(id="textarea7a", target="_blank", style={'font-size': '12px'}),
                    html.P(id="textarea7b", style={'color': colors['text'], 'font-size': '11px'}),
                    html.A(id="textarea8a", target="_blank", style={'font-size': '12px'}),
                    html.P(id="textarea8b", style={'color': colors['text'], 'font-size': '11px'}),
                    html.A(id="textarea9a", target="_blank", style={'font-size': '12px'}),
                    html.P(id="textarea9b", style={'color': colors['text'], 'font-size': '11px'}),
                    html.A(id="textarea10a", target="_blank", style={'font-size': '12px'}),
                    html.P(id="textarea10b", style={'color': colors['text'], 'font-size': '11px'}),
                    html.A(id="textarea11a", target="_blank", style={'font-size': '12px'}),
                    html.P(id="textarea11b", style={'color': colors['text'], 'font-size': '11px'}),
                    html.A(id="textarea12a", target="_blank", style={'font-size': '12px'}),
                    html.P(id="textarea12b", style={'color': colors['text'], 'font-size': '11px'}),
                    html.A(id="textarea13a", target="_blank", style={'font-size': '12px'}),
                    html.P(id="textarea13b", style={'color': colors['text'], 'font-size': '11px'}),
                    html.A(id="textarea14a", target="_blank", style={'font-size': '12px'}),
                    html.P(id="textarea14b", style={'color': colors['text'], 'font-size': '11px'}),
                    html.A(id="textarea15a", target="_blank", style={'font-size': '12px'}),
                    html.P(id="textarea15b", style={'color': colors['text'], 'font-size': '11px'})
                                                            
                ],
                     style={'backgroundColor': colors['blocks'],
                            'padding': '20px',
                            'border-radius': '5px',
                            'box-shadow': '1px 1px 1px #9D9D9D'},
                     className='one-half column'),
            
            html.Div(
                [
                    html.H6("Graphic summary",
                            style={'color': colors['titles']}),

                    html.Div([
                         dcc.Graph(id='graph1', style={'height': '300px'})
                         ],
                         style={'backgroundColor': colors['blocks'],
                                'padding': '20px'}
                    ),
                    
                    html.Div([
                         dcc.Graph(id='graph2', style={'height': '300px'})
                         ],
                         style={'backgroundColor': colors['blocks'],
                                'padding': '20px'}
                    )
                ],
                     style={'backgroundColor': colors['blocks'],
                            'padding': '20px',
                            'border-radius': '5px',
                            'box-shadow': '1px 1px 1px #9D9D9D'},
                     className='one-half column')

        ],
        className="row flex-display",
        style={'padding': '20px',
               'margin-bottom': '0px'}
    ),

    html.H1(id='space2', children=' '),
        

    html.Div(
            [dcc.Markdown(children=markdown_text2),],
            style={'font-size': '12px',
                   'color': colors['text']}),

    

    html.Div(id='intermediate-value', style={'display': 'none'})
    

])


@app.callback(
    [
    Output('intermediate-value', 'children'),
    Output('loading-1', 'children')
    ],
    [Input('submit', 'n_clicks')],
    [State('checklist', 'value')])
def scrape_and_predict(n_clicks, values):
            
    df_features = pd.DataFrame()
    df_show_info = pd.DataFrame()
    

    
    if 'THG' in values:
        df_features = df_features.append(get_news_theguardian()[0])
        df_show_info = df_show_info.append(get_news_theguardian()[1])
    if 'DML' in values:
        df_features = df_features.append(get_news_dailymail()[0])
        df_show_info = df_show_info.append(get_news_dailymail()[1])

    df_features = df_features.reset_index().drop('index', axis=1)
    
    
    features = create_features_from_df(df_features)
    predictions = predict_from_features(features)
    df = complete_df(df_show_info, predictions)
    
    return df.to_json(date_format='iso', orient='split'), ' '

@app.callback(
    Output('graph1', 'figure'),
    [Input('intermediate-value', 'children')])
def update_barchart(jsonified_df):
    
    df = pd.read_json(jsonified_df, orient='split')
    df_sum = df.groupby(['Newspaper', 'Prediction']).count()['Article Title']
    
    if 'The Guardian' in df_sum.index:
        
        df_sum_thg = df_sum['The Guardian']
        x_thg = ['Politics', 'Business', 'Entertainment', 'Sport', 'Tech', 'Other']
        y_thg = [[df_sum_thg['politics'] if 'politics' in df_sum_thg.index else 0][0],
                [df_sum_thg['business'] if 'business' in df_sum_thg.index else 0][0],
                [df_sum_thg['entertainment'] if 'entertainment' in df_sum_thg.index else 0][0],
                [df_sum_thg['sport'] if 'sport' in df_sum_thg.index else 0][0],
                [df_sum_thg['tech'] if 'tech' in df_sum_thg.index else 0][0],
                [df_sum_thg['other'] if 'other' in df_sum_thg.index else 0][0]]   
    else:
        x_thg = ['Politics', 'Business', 'Entertainment', 'Sport', 'Tech', 'Other']
        y_thg = [0,0,0,0,0,0]

    if 'Daily Mail' in df_sum.index:
    
        df_sum_skn = df_sum['Daily Mail']
        x_skn = ['Politics', 'Business', 'Entertainment', 'Sport', 'Tech', 'Other']
        y_skn = [[df_sum_skn['politics'] if 'politics' in df_sum_skn.index else 0][0],
                [df_sum_skn['business'] if 'business' in df_sum_skn.index else 0][0],
                [df_sum_skn['entertainment'] if 'entertainment' in df_sum_skn.index else 0][0],
                [df_sum_skn['sport'] if 'sport' in df_sum_skn.index else 0][0],
                [df_sum_skn['tech'] if 'tech' in df_sum_skn.index else 0][0],
                [df_sum_skn['other'] if 'other' in df_sum_skn.index else 0][0]]   

    else:
        x_skn = ['Politics', 'Business', 'Entertainment', 'Sport', 'Tech', 'Other']
        y_skn = [0,0,0,0,0,0]

    figure = {
        'data': [
            
            {'x': x_thg, 'y':y_thg, 'type': 'bar', 'name': 'The Guardian', 'marker': {'color': 'rgb(167, 203, 232)'}},
            {'x': x_skn, 'y':y_skn, 'type': 'bar', 'name': 'Daily Mail', 'marker': {'color': 'rgb(197, 223, 242)'}}
        ],
        'layout': {
            'title': 'Number of news articles by newspaper',
            'plot_bgcolor': colors['graph_background'],
            'paper_bgcolor': colors['graph_background'],
            'font': {
                    'color': colors['text'],
                    'size': '10'
            },
            'barmode': 'stack'
            
        }   
    }

    return figure

@app.callback(
    Output('graph2', 'figure'),
    [Input('intermediate-value', 'children')])
def update_piechart(jsonified_df):
    
    df = pd.read_json(jsonified_df, orient='split')
    df_sum = df['Prediction'].value_counts()


    x = ['Politics', 'Business', 'Entertainment', 'Sport', 'Tech', 'Other']
    y = [[df_sum['politics'] if 'politics' in df_sum.index else 0][0],
         [df_sum['business'] if 'business' in df_sum.index else 0][0],
         [df_sum['entertainment'] if 'entertainment' in df_sum.index else 0][0],
         [df_sum['sport'] if 'sport' in df_sum.index else 0][0],
         [df_sum['tech'] if 'tech' in df_sum.index else 0][0],
         [df_sum['other'] if 'other' in df_sum.index else 0][0]]
    

    figure = {
        'data': [
            {'values': y,
             'labels': x, 
             'type': 'pie',
             'hole': .4,
             'name': '% of news articles',
             'marker': {'colors': ['rgb(62, 137, 195)',
                                   'rgb(167, 203, 232)',
                                   'rgb(197, 223, 242)',
                                   'rgb(51, 113, 159)',
                                   'rgb(64, 111, 146)',
                                   'rgb(31, 84, 132)']},

            }
        ],
        
        'layout': {
            'title': 'News articles by newspaper',
            'plot_bgcolor': colors['graph_background'],
            'paper_bgcolor': colors['graph_background'],
            'font': {
                    'color': colors['text'],
                    'size': '10'
            }
        }
        
    }
    
    return figure
    
@app.callback(
    [
    Output('textarea1a', 'href'),
    Output('textarea1a', 'children'),
    Output('textarea1b', 'children'),
    Output('textarea2a', 'href'),
    Output('textarea2a', 'children'),
    Output('textarea2b', 'children'),
    Output('textarea3a', 'href'),
    Output('textarea3a', 'children'),
    Output('textarea3b', 'children'),
    Output('textarea4a', 'href'),
    Output('textarea4a', 'children'),
    Output('textarea4b', 'children'),
    Output('textarea5a', 'href'),
    Output('textarea5a', 'children'),
    Output('textarea5b', 'children'),
    Output('textarea6a', 'href'),
    Output('textarea6a', 'children'),
    Output('textarea6b', 'children'),
    Output('textarea7a', 'href'),
    Output('textarea7a', 'children'),
    Output('textarea7b', 'children'),
    Output('textarea8a', 'href'),
    Output('textarea8a', 'children'),
    Output('textarea8b', 'children'),
    Output('textarea9a', 'href'),
    Output('textarea9a', 'children'),
    Output('textarea9b', 'children'),
    Output('textarea10a', 'href'),
    Output('textarea10a', 'children'),
    Output('textarea10b', 'children'),
    Output('textarea11a', 'href'),
    Output('textarea11a', 'children'),
    Output('textarea11b', 'children'),
    Output('textarea12a', 'href'),
    Output('textarea12a', 'children'),
    Output('textarea12b', 'children'),
    Output('textarea13a', 'href'),
    Output('textarea13a', 'children'),
    Output('textarea13b', 'children'),
    Output('textarea14a', 'href'),
    Output('textarea14a', 'children'),
    Output('textarea14b', 'children'),
    Output('textarea15a', 'href'),
    Output('textarea15a', 'children'),
    Output('textarea15b', 'children')
    ],
    [Input('intermediate-value', 'children')])
def update_textarea1(jsonified_df):
    
    df = pd.read_json(jsonified_df, orient='split')
    
    texts = []
    links = []
    preds_newsp = []
    
    for article in range(len(df)):
        texts.append(df.iloc[article]['Article Title'])
        links.append(df.iloc[article]['Article Link'])
        preds_newsp.append((df.iloc[article]['Prediction'].capitalize()) + ', ' + (df.iloc[article]['Newspaper']))

    while (len(texts) < 16):
        texts.append(None)
        links.append(None)
        preds_newsp.append(None)
    
    return \
        links[0], texts[0], preds_newsp[0],\
        links[1], texts[1], preds_newsp[1],\
        links[2], texts[2], preds_newsp[2],\
        links[3], texts[3], preds_newsp[3],\
        links[4], texts[4], preds_newsp[4],\
        links[5], texts[5], preds_newsp[5],\
        links[6], texts[6], preds_newsp[6],\
        links[7], texts[7], preds_newsp[7],\
        links[8], texts[8], preds_newsp[8],\
        links[9], texts[9], preds_newsp[9],\
        links[10], texts[10], preds_newsp[10],\
        links[11], texts[11], preds_newsp[11],\
        links[12], texts[12], preds_newsp[12],\
        links[13], texts[13], preds_newsp[13],\
        links[14], texts[14], preds_newsp[14]
           
    
    

app.css.append_css({"external_url": "https://codepen.io/chriddyp/pen/bWLwgP.css"})
app.css.append_css({"external_url": "https://codepen.io/chriddyp/pen/brPBPO.css"})

In [ ]:
app.run_server(debug=False)